In [1]:
import random
from gx_utils import *
import numpy as np

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
import logging


def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [4]:

def depth_first(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    frontier = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    def build_solution_recursive(solution,covered,all_lists):
        #return solution
        #print(f"solution:{solution}")
        if goal == covered: #terminal condition
            #print("found!")
            return solution
        for i,x in enumerate(all_lists):
            next_all_lists = all_lists.copy() #create a copy of the list[list]
            next_list = next_all_lists.pop(i) #then pop the next list
            if not set(next_list) < covered: #if next list add some new number to my solution
                #make a (shallow) copy of my state and update that one
                next_covered = covered.copy()
                next_covered |= set(next_list)
                next_solution = solution.copy()
                next_solution.append(next_list)
                return build_solution_recursive(next_solution,next_covered,next_all_lists)
        return list() #no solution

    solution = build_solution_recursive(solution,covered,all_lists)
    logging.info(
        f"depth first solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")




In [5]:

def breadth_first(N):
    goal = set(range(N))
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    def build_solution(goal, all_lists):
        node_visited = 0
        covered = set()
        solution = list()
        frontier = list() # ( solution , covered, all_lists)
        #first step needed for initial state in recursive function
        for i,x in enumerate(all_lists):
                next_all_lists = all_lists.copy() #create a copy of the list[list]
                next_list = next_all_lists.pop(i) #then pop the next list
                if not set(next_list) < covered: #if next list add some new number to my solution
                    #make a (shallow) copy of my state and update that one
                    next_covered = covered.copy()
                    next_covered |= set(next_list)
                    next_solution = solution.copy()
                    next_solution.append(next_list)
                    frontier.append((next_solution,next_covered,next_all_lists))
        while len(frontier):
            (solution,covered,all_lists) = frontier.pop(0)
            node_visited +=1
            if covered == goal:
                return (solution,node_visited)
            for i,x in enumerate(all_lists):
                next_all_lists = all_lists.copy() #create a copy of the list[list]
                next_list = next_all_lists.pop(i) #then pop the next list
                if not set(next_list) < covered: #if next list add some new number to my solution
                    #make a (shallow) copy of my state and update that one
                    next_covered = covered.copy()
                    next_covered |= set(next_list)
                    next_solution = solution.copy()
                    next_solution.append(next_list)
                    frontier.append((next_solution,next_covered,next_all_lists))
        return (list(),node_visited)
        
    

    (solution,node_visited) = build_solution(goal, all_lists)

    logging.info(
        f"breadth first solution for N={N}: visiting {node_visited} nodes, w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")

In [38]:

def a_star(N):

    def total_cost(state):
        g = state[0] #w : the weight of the node
        h = N - len(state[1]) #number of element missing to reach the solution
        return g + h

    node_visited = 0
    goal = set(range(N))
    covered = frozenset()
    solution = list()
    frontier = PriorityQueue()
    state = (0,covered) # ( w , set )
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    cost = lambda s: total_cost(s) #function to compute the cost
    for x_list in all_lists: #initial state, popolate the frontier
        new_state = (len(x_list),frozenset(x_list))
        if new_state not in frontier:
            frontier.push(new_state,p = cost(new_state))

    state = frontier.pop() #pop the first one
    node_visited += 1
    

    while state[1] != goal and frontier is not None:
        for x_list in all_lists:
            if frozenset(x_list) not in state[1]: #the list add some coverage to the current state
                new_state = (state[0]+len(x_list),state[1] | frozenset(x_list)) #create a new state with the list added
                if new_state not in frontier: #if the state is not already in the frontier
                    frontier.push(new_state,p = cost(new_state)) #add the state to the frontier
        state = frontier.pop() #take the best one from the frontier  
        node_visited += 1  #and increment the number of visited node

    logging.info(
    f"A* solution for N={N} visiting {node_visited} nodes: w={state[0]} (bloat={(state[0]-N)/N*100:.0f}%)"
    )
    logging.debug(f"{state[1]}")
    return (state)
 

In [39]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20]: #100 or more are not fisible in a reasonable amount of time
    a_star(N)

INFO:root:A* solution for N=5 visiting 31 nodes: w=5 (bloat=0%)
INFO:root:A* solution for N=10 visiting 484 nodes: w=10 (bloat=0%)
INFO:root:A* solution for N=20 visiting 3394 nodes: w=23 (bloat=15%)
